In [6]:
import os
from dotenv import load_dotenv
from langchain_community.agent_toolkits import GmailToolkit
from langchain_community.tools.gmail.utils import build_resource_service,get_gmail_credentials
from langchain import hub
from langchain.agents import AgentExecutor,create_openai_functions_agent
import google.generativeai as genai


In [2]:
load_dotenv()

True

In [3]:
os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

In [4]:
model=genai.GenerativeModel("gemini-1.5-flash")
response=model.generate_content('how is the weather in my current location?')
print(response.text)

I do not have access to your current location. To get the weather in your location, you can use a weather app on your phone or computer, or search for "[your location] weather" on the internet.



In [5]:
# set up Gmail API credentials
credential=get_gmail_credentials(token_file='token.json',client_secrets_file='client_secret.json', 
                       scopes=['https://mail.google.com/'])

#build api resource service for gmail
api_resource=build_resource_service(credentials=credential)

#initialize gmail toolkit
gmail_toolkit=GmailToolkit(api_resource=api_resource)
                                 
tools=gmail_toolkit.get_tools()                                 

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=848333861849-4nd3ombc70mqa9oiuof54hco1l5fgkcv.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53047%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=UsmAPTxHyeL5WaytbGRv4VPhmnuWaD&access_type=offline


In [ ]:

#pull the base prompt template from langchain hub
template=hub.pull("hwchase17/openai-chat-agent", "base_prompt.txt")

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI   

In [10]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0,api_key=os.getenv('GEMINI_API_KEY'))

In [13]:
#pull the base prompt template from langchain hub
base_prompt=hub.pull('langchain-ai/openai-functions-template')

In [12]:
instruction = (
    "You are a helpful email assistant agent that manages emails. "
    "You can read, send, and organize emails using the tools provided. "
    "Always respond concisely, professionally, and without exposing sensitive information."
)

In [14]:
prompt=base_prompt.partial(instruction=instruction)

In [15]:
# create the agent
agent=create_openai_functions_agent(llm=llm,prompt=prompt,tools=tools)

In [16]:
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=False) #set verbose to false to avoid the sensitive info being printed

In [ ]:
# invoke the agent to draft an email
response=agent_executor.invoke({'input':'draft an email regarding project update to mia',
                                'instructions':'inform mia that the project is on track and will be completed by the end of the month along add the corresponding subject , write the professional email it should be in a proper structure with greeting and sign off'})

In [26]:
for detail in response.items():
    print(detail)

('input', 'draft an email regarding project update to mia')
('instructions', 'inform mia that the project is on track and will be completed by the end of the month along add the corresponding subject , write the professional email it should be in a proper structure with greeting and sign off')
('output', 'OK. I\'ve drafted an email to Mia.  The draft has been saved with the subject "Project Update".  The email informs Mia that the project is on track for completion by the end of the month.  Do you want me to send it, or do you need to make any changes to the draft before sending?')


In [37]:
# invoke the agent to check total counts of today
response=agent_executor.invoke({'input':'how many emails do I have in my inbox today 3-sep-2025',
                                'instructions':'Check in my primary inbox and give me the total count of emails received today'})
print(response)

{'input': 'how many emails do I have in my inbox today 3-sep-2025', 'instructions': 'Check in my primary inbox and give me the total count of emails received today', 'output': 'You received 10 emails in your primary inbox today, September 3, 2025.'}


In [30]:
# invoke the agent to draft an email
response=agent_executor.invoke({'input':'send an email from (meenamgrp@gmail.com) email to meenakshi (meenamdatascientist@gmail.com) regarding the project update',
                                'instructions':'email should be regarding the project update of new client need to know whether the project is completed or not and should be professional in tone and should have a proper structure with greeting and sign off along with the subject '})
print(response)

{'input': 'send an email from (meenamgrp@gmail.com) email to meenakshi (meenamdatascientist@gmail.com) regarding the project update', 'instructions': 'email should be regarding the project update of new client need to know whether the project is completed or not and should be professional in tone and should have a proper structure with greeting and sign off along with the subject ', 'output': "OK. I've sent the email.  Is there anything else?"}


In [38]:
# invoke the agent to draft an email
response=agent_executor.invoke({'input':'who is the sender of the latest email in my inbox today 3-sep-2025',
                                'instructions':'Check in my primary inbox and give me the details of the latest email'})
print(response)

{'input': 'who is the sender of the latest email in my inbox today 3-sep-2025', 'instructions': 'Check in my primary inbox and give me the details of the latest email', 'output': 'The sender of the latest email in your inbox on September 3rd, 2025 is IBM via LinkedIn.  The email is a newsletter.'}


In [44]:
# invoke the agent to draft an email
response=agent_executor.invoke({'input':'send an email from meena (meenamgrp@gmail.com) email to meenakshi (meenamdatascientist@gmail.com),mia (meena15grp@gmail.com) regarding the project new task',
                   'instructions':'compose email for two recipients  regarding the project new task. meenakshi (meenamdatascientist@gmail.com) need to work on data cleaning,model building ,deploying and mia (meena15grp@gmail.com) need to work on data collection,data analysis and data visualization.along with that need to ask any doubt with their task.send exact tasks to their email. email should be professional maximum 7 lines,properly aligned space, greeting and sign off along with the subject as their coincise task and mark the important things in bold and italics'})
print(response)

{'input': 'send an email from meena (meenamgrp@gmail.com) email to meenakshi (meenamdatascientist@gmail.com),mia (meena15grp@gmail.com) regarding the project new task', 'instructions': 'compose email for two recipients  regarding the project new task. meenakshi (meenamdatascientist@gmail.com) need to work on data cleaning,model building ,deploying and mia (meena15grp@gmail.com) need to work on data collection,data analysis and data visualization.along with that need to ask any doubt with their task.send exact tasks to their email. email should be professional maximum 7 lines,properly aligned space, greeting and sign off along with the subject as their coincise task and mark the important things in bold and italics', 'output': 'OK. I have sent a draft email.  Please check your Gmail drafts.'}
